In [14]:
# !git clone https://github.com/AlexeyAB/darknet.git

In [16]:
!pwd
%cd /home/taozang2/darknet

/home/taozang2/darknet
/home/taozang2/darknet


In [17]:
!make # already make

chmod +x *.sh
nvcc -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_50,code=[sm_50,compute_50] -gencode arch=compute_52,code=[sm_52,compute_52] -gencode arch=compute_61,code=[sm_61,compute_61] -gencode arch=compute_70,code=[sm_70,compute_70] -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF --compiler-options "-Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF" -c ./src/convolutional_kernels.cu -o obj/convolutional_kernels.o
/bin/sh: nvcc: command not found
make: *** [obj/convolutional_kernels.o] Error 127


In [20]:
%cd dataset1026/ # latest datset available
!ls

/home/taozang2/darknet/dataset1026
classes.txt
image-0_42.150377999999996_-87.883777.jpg
image-0_42.150377999999996_-87.883777.txt
image-100_41.666525_-87.651152.jpg
image-100_41.666525_-87.651152.txt
image-101_42.07604_-87.84140099999999.jpg
image-101_42.07604_-87.84140099999999.txt
image-102_42.07929_-87.83369699999999.jpg
image-102_42.07929_-87.83369699999999.txt
image-103_42.147777999999995_-87.891481.jpg
image-103_42.147777999999995_-87.891481.txt
image-10_41.663925_-87.65457599999999.jpg
image-10_41.663925_-87.65457599999999.txt
image-104_41.665225_-87.658.jpg
image-104_41.665225_-87.658.txt
image-105_42.07669_-87.849961.jpg
image-105_42.07669_-87.849961.txt
image-106_41.662625_-87.652864.jpg
image-106_41.662625_-87.652864.txt
image-107_41.665875_-87.65457599999999.jpg
image-107_41.665875_-87.65457599999999.txt
image-108_42.146477999999995_-87.888057.jpg
image-108_42.146477999999995_-87.888057.txt
image-109_41.668475_-87.64516.jpg
image-109_41.668475_-87.64516.txt
image-110_42.14

In [45]:
PATH = '/home/taozang2/darknet/dataset1026/'

In [46]:
import numpy as np
import os
from shutil import copy
from sklearn.model_selection import train_test_split

def get_image_list(path):
    filenames = os.listdir(PATH)
    # store the names for images and labels
    img = []
    lbl = []
    for file in filenames: 
        if file.endswith('.jpg'):
            img.append(file)
        if file.endswith('.txt'):
            lbl.append(file)
    assert len(img) == len(lbl)
    img = np.asarray(img)
    lbl = np.asarray(lbl)
    return img, lbl


    

In [48]:
# test get_image_list
images, annotations = get_image_list(PATH)
N = images.shape
N[0] / 5 # discard two images with only 1 bounding box

108.0

In [50]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)

In [70]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
import os
cv_path = '/home/taozang2/darknet/cross_validation/'
N = kf.get_n_splits()
n = 0

for i in range(N):
    os.makedirs(cv_path + '%d/obj'%i)
    os.makedirs(cv_path + '%d/test'%i)

In [71]:
for train_index, test_index in kf.split(images):
    img_train, img_test = images[train_index], images[test_index]
    
    # split images into train and test folders
    for img in img_train:
        label = img.split('.jpg')[0] + '.txt'
        copy(PATH + img, cv_path + '%d/obj'%n)
        copy(PATH + label, cv_path + '%d/obj'%n)
    
    for img in img_test:
        label = img.split('.jpg')[0] + '.txt'
        copy(PATH + img, cv_path + '%d/test'%n)
        copy(PATH + label, cv_path +'%d/test'%n)
    n = n + 1